<a href="https://colab.research.google.com/github/AnikethDandu/traffic-sign-classification/blob/main/TrafficSignClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Traffic Sign Classification**


## **Import Libraries**

In [26]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

## **Placeholder**

In [27]:
EPOCHS = 10
BATCH_SIZE = 16
learning_rate = 0.001

image_size = (-1, 3, 50, 50)

training_dataset = None
testing_dataset = None
train_dataloader = None
test_dataloader = None

train_path = 'traffic_sign_images/Train'

## **Convolutional Neural Network**

In [28]:
class ConvNet(nn.Module):
  def __init__(self):
    # Input image: 32x32x3
    self.PADDING_SIZE = 1
    self.KERNEL_SIZE = 3
    self.STRIDE = 1
    self.POOL_SIZE = 2
    super().__init__()
    self.conv1 = nn.Conv2d(3, 32, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv2 = nn.Conv2d(32, 64, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv3 = nn.Conv2d(64, 128, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv4 = nn.Conv2d(128, 256, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.fc1 = nn.Linear(2304, 512)
    self.fc2 = nn.Linear(512, 43)
    
  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv2(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv3(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv4(x)), self.POOL_SIZE)
    x = x.flatten(start_dim=1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x 

## **Dataset**

### **Custom Dataset Class**

In [29]:
class TrafficSignDataset(Dataset):
  def __init__(self, train, root_dir, img_size):
    self.train = train
    self.root_dir = root_dir
    self.df = pd.read_csv(os.path.join(root_dir, 'Train.csv' if train else 'Test.csv'))
    self.img_size = img_size

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    image = cv2.imread(os.path.join(self.root_dir, self.df.iloc[idx][7]), cv2.IMREAD_COLOR)
    image = cv2.resize(image, (self.img_size, self.img_size))
    label = self.df.iloc[idx][6]
    sample = {'image': torch.tensor(image), 'label': self.df.iloc[idx][6]}
    return sample


### **Dataset creation function**

In [30]:
def create_datasets():
  global training_dataset
  global testing_dataset
  global train_dataloader
  global test_dataloader

  training_dataset = TrafficSignDataset(train=True, root_dir='traffic_sign_images', img_size=50)
  testing_dataset = TrafficSignDataset(train=False, root_dir='traffic_sign_images', img_size=50)
  
  train_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
  test_dataloader = DataLoader(testing_dataset, batch_size=1, shuffle=True)


## **Model Training and Evaluation**

### **Model Training Function**

In [37]:
def train_model(net):
  for epoch in range(EPOCHS):
    for batch_idx, batch in enumerate(train_dataloader):
      batch_imgs, batch_lbls = batch["image"].view(image_size) / 255.0, batch["label"]
      batch_labels = [0 for i in range(BATCH_SIZE)]
      for label_idx, label in enumerate(batch_lbls):
        batch_labels[label_idx] = label.item()
        
      batch_imgs = batch_imgs
      batch_labels = batch_labels
      
      optimizer.zero_grad()
      outputs = net(batch_imgs.to(device))
      loss = criterion(outputs, torch.tensor([label for label in batch_labels], device=device).long())
      loss.backward()
      optimizer.step()
    print(f'Epoch: {epoch + 1}, Loss: {loss}')

### **Model Evaluation Function**

In [32]:
def evaluate_model(net):
  total_classes = {}
  class_correct = {}
  total_images = 0
  total_correct = 0
  with torch.no_grad():
    for batch_idx, batch in enumerate(test_dataloader):
      test_image, test_label = batch['image'].view(image_size) / 255.0, batch['label'].item()
      correct_class = test_label
      test_image = test_image.to(device)
      predicted_class = torch.argmax(net(test_image)[0])
      
      total_images += 1
      total_classes[predicted_class.item()] = total_classes[predicted_class.item()] + 1 if predicted_class.item() in total_classes else 1
      
      if predicted_class == correct_class:
        total_correct += 1
        class_correct[correct_class] = class_correct[correct_class] + 1 if correct_class in class_correct else 1
  print([f'Accuracy for {img_class}: {round(100 * class_correct[img_class] / total_classes[img_class], 3)}%' for img_class in class_correct])
  print(f'Raw class score: {class_correct}')
  print(f'Total images correct: {total_correct}, Total images: {total_images}, Total accuracy: {round(100 * total_correct / total_images, 3)}%')

## **Main Script**

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

create_datasets()

total_images = 0
class_count = []
for folder in os.listdir(train_path):
  if folder != '.DS_Store':
    image_count = len([img for img in os.listdir(os.path.join(train_path, folder))])
    class_count.append(image_count)
    total_images += image_count

final_weights = torch.Tensor([1 - img_count/total_images for img_count in class_count]).to(device)
conv_net = ConvNet().to(device)
optimizer = optim.Adam(conv_net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(weight=final_weights)

train_model(conv_net)
evaluate_model(conv_net)

## **Google Drive Dataset Import**

In [ ]:
'''
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r /content/gdrive/My\ Drive/ColabNotebooks/Data/ traffic_sign_images.zip
!unzip traffic_sign_images.zip/traffic_sign_images.zip
!rm -r traffic_sign_images.zip/
'''